### SERPAPI 

In [1]:
import numpy as np
import pandas as pd

from datetime import datetime
from time import sleep

import pickle as pkl

from pprint import pprint
from libraries.serpapi import serpapi

In [2]:
# Cargamos los puestos y nuestra API_KEY
today = datetime.now().strftime("%Y-%m-%d")

with open(file = "serpapi_key.txt", mode = "r") as file:
    api_key = file.read()

lista_puestos = pd.read_excel("data/lista_puestos.xlsx", header = None)
lista_puestos.columns = ["jobs"]
lista_puestos

,jobs
0,Desarrollador de software
1,Programador web
2,Consultor tecnológico
3,Ingeniero de sistemas
4,Especialista en seguridad informática
...,...
478,Desarrollador de software científico
479,Técnico de soporte remoto
480,Especialista en redes informáticas
481,Analista de seguridad en la nube


In [3]:
with open("data/lista_puestos_ingles.txt", "r") as file:
    lista_puestos_ingles = file.read()
    
lista_puestos_ingles = lista_puestos_ingles.split("\n")

lista_puestos_ingles = [x for x in [x.replace("Developer", "") for x in lista_puestos_ingles] if len(x.split()) > 1]

In [4]:
# Ajustamos España como país de búsqueda
spain = [[".es", "Spain"]]

# Ajustamos Latino-américa como países de búsqueda
latam = [["com.ar", "Argentina"]         ,
         ["com.bo", "Bolivia"]           ,
         ["com.br", "Brazil"]            ,
         ["cl", "Chile"]             ,
         ["com.co", "Colombia"]          ,
         ["co.cr", "Costa Rica"]        ,
         ["com.cu", "Cuba"]              ,
         ["com.do", "Dominican Republic"],
         ["com.ec", "Ecuador"]           ,
         ["com.sv", "El Salvador"]       ,
         ["com.gt", "Guatemala"]         ,
         ["hn", "Honduras"]          ,
         ["com.mx", "Mexico"]            ,
         ["com.ni", "Nicaragua"]         ,
         ["com.pa", "Panama"]            ,
         ["com.py", "Paraguay"]          ,
         ["com.pe", "Peru"]              ,
         ["com.pr", "Puerto Rico"]       ,
         ["com.uy", "Uruguay"]]

In [5]:
%%time

df = pd.DataFrame()

for abr, country in spain:
    
    for q in lista_puestos_ingles: # Modificar lista de puestos según país/países de preferencia.

        try:
            
            for pagination in range(100):
                
                print(f"{q:60}{pagination}")

                q_params = {"q"             : q,
                            "api_key"       : api_key,
                            "location"      : country.title(),
                            "start"         : pagination*10
                            }

                response = serpapi.job_search(**q_params)


                if ("error" in response) or (response.get("jobs_results") == None) or (len(response.get("jobs_results")) < 10):
                    break

                df_response = pd.json_normalize(response["jobs_results"])

                df_response.columns = [x.split(".")[0] if len(x.split(".")) == 1 else x.split(".")[-1] for x in df_response.columns]

                df_response["country_search"] = country.title()
                
                df = pd.concat(objs = [df, df_response], ignore_index = True)
                
        except:
            
            print(f"Error {q} ***************************************************************")
                 
df = df.drop_duplicates(subset = "job_id").reset_index(drop = True)
df["date_posted"] = datetime.strptime(today, "%Y-%m-%d").date()

df.to_csv(f"data/extraction_{today}_latam.csv", index = False)

Web Programmer                                              0
Web Programmer                                              1
Web Programmer                                              2
Technology Consultant                                       0
Technology Consultant                                       1
Technology Consultant                                       2
Systems Engineer                                            0
Systems Engineer                                            1
Systems Engineer                                            2
Systems Engineer                                            3
Systems Engineer                                            4
Information Security Specialist                             0
Programmer Analyst                                          0
Mobile App                                                  0
Mobile App                                                  1
Mobile App                                                  2
Mobile A

Software Engineer                                           5
Software Engineer                                           6
Software Engineer                                           7
Software Engineer                                           8
Software Engineer                                           9
Software Engineer                                           10
Software Engineer                                           11
Software Engineer                                           12
Software Engineer                                           13
Big Data Consultant                                         0
Big Data Consultant                                         1
Big Data Consultant                                         2
Systems Administrator                                       0
Systems Administrator                                       1
Cloud Computing Specialist                                  0
Cloud Computing Specialist                                  1
Clou

Python Software                                             2
Python Software                                             3
Python Software                                             4
Python Software                                             5
Python Software                                             6
Python Software                                             7
Python Software                                             8
Python Software                                             9
Python Software                                             10
Data Engineer                                               0
Data Engineer                                               1
Data Engineer                                               2
Data Engineer                                               3
Data Engineer                                               4
Data Engineer                                               5
Data Engineer                                               6
Data En

Data Analysis Specialist                                    16
ERP Systems Specialist                                      0
Automation Engineer                                         0
Automation Engineer                                         1
Automation Engineer                                         2
Automation Engineer                                         3
Automation Engineer                                         4
Automation Engineer                                         5
Automation Engineer                                         6
Automation Engineer                                         7
Automation Engineer                                         8
Automation Engineer                                         9
Automation Engineer                                         10
Automation Engineer                                         11
Automation Engineer                                         12
Automation Engineer                                         13
Ora

Business Analyst                                            0
Business Analyst                                            1
Business Analyst                                            2
Business Analyst                                            3
Business Analyst                                            4
Business Analyst                                            5
Business Analyst                                            6
Business Analyst                                            7
Business Analyst                                            8
Business Analyst                                            9
Business Analyst                                            10
Business Analyst                                            11
Microservices Specialist                                    0
Physical Security Specialist                                0
Emotional Intelligence Specialist                           0
Geographic Information Systems Specialist                   0
Cloud 

...

Mobile Device Repair Technician                             0
Wireless Network Security Specialist                        0
SCADA Systems Specialist                                    0
Relational Database Specialist                              0
Systems Analyst                                             0
IT Project Coordinator                                      0
Cybersecurity Consultant                                    0
Embedded Systems Software                                   0
Java Programmer                                             0
Enterprise Management Systems Expert                        0
Embedded Systems Software Engineer                          0
Technical Support Coordinator                               0
C++ Programmer                                              0
Telecommunications Systems Technician                       0
Distributed Systems Software Engineer                       0
Information Systems Consultant                              0
Game Sof

COBOL Programmer                                            0
Process Control Systems Specialist                          0
Application Support Technician                              0
Enterprise Information Security Specialist                  0
Oracle Database Administrator                               0
UNIX Systems Technician                                     0
Sales Data Analyst                                          0
Business AI Specialist                                      0
Software Testing Automation Specialist                      0
Mobile Solutions                                            0
Hospital Information Systems Specialist                     0
AS400 Systems Technician                                    0
CRM Software Development Specialist                         0
Quality Data Analyst                                        0
Scientific Software                                         0
Remote Support Technician                                   0
Computer

Systems Integration Specialist                              0
Unity Programmer                                            0
Firmware Programmer                                         0
Juniper Network Technician                                  0
NetApp Storage Systems Specialist                           0
Oracle Support Technician                                   0
ABAP Software                                               0
Unix/Linux Operating Systems Specialist                     0
Software Quality Specialist                                 0
Microsoft Support Technician                                0
SharePoint Programmer                                       0
Huawei Network Technician                                   0
Windows Systems Engineer                                    0
VMWare Support Technician                                   0
Salesforce Support Technician                               0
SharePoint Software                                         0
Aruba Ne

Applied Artificial Intelligence Specialist in Medicine      0
Hybrid Mobile App Programmer                                0
Augmented Reality Specialist                                0
Data Analysis Software                                      0
Remote IoT Specialist                                       0
Security Systems Engineer                                   0
Network Automation Specialist                               0
Computer Maintenance Technician                             0
Communications Network Support Technician                   0
Computer Security Technician                                0
Back-end Software                                           0
Network and Systems Technician                              0
IT Security Consultant                                      0
UX Writing Specialist                                       0
Google Analytics Specialist                                 0
AdWords Specialist                                          0
Cybersec

ERP Software                                                0
CRM Software                                                0
Web Application Security Specialist                         0
Electronic Payment Systems Specialist                       0
RFID Technology Specialist                                  0
Chatbot Development Specialist                              0
Cloud Solutions                                             0
Automotive Software                                         0
Aerospace Industry Systems                                  0
Process Control Systems                                     0
Hybrid Mobile App                                           0
Software Quality Technician                                 0
Software Maintenance Technician                             0
Robotics Software                                           0
Mobile Cybersecurity Specialist                             0
IoT Software                                                0
Network 

In [ ]:
################################################################################################################################